In [1]:
import requests
import json
import math
import csv

In [2]:
# set key
key="c1d0b0be71524b30bdd89e1c27098eee"

# set base url
base_url="http://api.nytimes.com/svc/search/v2/articlesearch"

# set response format
response_format=".json"

# set parameters
search_params = {"q":"Trump + Healthcare",
                 "api-key":key,
                 "begin_date":"20160501", 
                 "end_date":"20170430"}

In [3]:
# make request
r = requests.get(base_url+response_format, params=search_params)

In [6]:
response_text= r.text
print(response_text[:1000])

{"status":"OK","copyright":"Copyright (c) 2018 The New York Times Company. All Rights Reserved.","response":{"docs":[{"web_url":"https://www.nytimes.com/2017/04/29/us/politics/fact-checking-president-trump-through-his-first-100-days.html","snippet":"In his first 100 days in office, President Trump has falsely boasted of attracting the largest inaugural crowd ever and leveled an unproved accusation that his predecessor spied on him.","print_page":"0","blog":{},"source":"The New York Times","multimedia":[{"rank":0,"subtype":"thumbnail","caption":null,"credit":null,"type":"image","url":"images/2017/04/29/us/29factcheck1/29factcheck1-thumbStandard.jpg","height":75,"width":75,"legacy":{"thumbnailheight":75,"thumbnail":"images/2017/04/29/us/29factcheck1/29factcheck1-thumbStandard.jpg","thumbnailwidth":75},"subType":"thumbnail","crop_name":null},{"rank":0,"subtype":"xlarge","caption":null,"credit":null,"type":"image","url":"images/2017/04/29/us/29factcheck1/29factcheck1-articleLarge.jpg","hei

In [7]:
print(r.url)

http://api.nytimes.com/svc/search/v2/articlesearch.json?q=Trump+%2B+Healthcare&api-key=c1d0b0be71524b30bdd89e1c27098eee&begin_date=20160501&end_date=20170430


In [107]:
# Convert JSON response to a dictionary because json text is similar to dictionary (key - value)
data = json.loads(response_text)


In [9]:
# dictionary is abt key value
print(data.keys())

dict_keys(['status', 'copyright', 'response'])


In [14]:
data['status']


'OK'

In [15]:
data['copyright']

'Copyright (c) 2018 The New York Times Company. All Rights Reserved.'

In [22]:
docs = data ['response'],['docs']

In [106]:
type(docs)

list

In [24]:
type(docs)

tuple

In [38]:
# To get the number of hits
hit = data['response']['meta']['hits']
print(hit)

76


In [55]:
# no of pages (10 hits per pages)
pages = int (math.ceil(hit/10))
pages

8

In [56]:
all_docs = []
for i in range(pages):
    print("page No %d" %i)   
    search_params ['page'] = i
    r = requests.get(base_url+response_format, params=search_params)
    data = json.loads(response_text)
    docs = data ['response']['docs']
    all_docs = all_docs + docs
    

page No 0
page No 1
page No 2
page No 3
page No 4
page No 5
page No 6
page No 7


In [57]:
len(all_docs)

80

In [98]:
def format_articles(unformatted_docs):
    formatted = []
    for i in unformatted_docs:
        dic = {}
        dic['id'] = i['_id']
        dic['headline'] = i['headline']['main']
        dic['date'] = i['pub_date'] [0:10]
        dic['source'] = i['source']
        dic['document_type'] = i['document_type']
        
        formatted.append(dic)
    return(formatted)

In [99]:
 all_formatted = format_articles(all_docs)


In [100]:
all_formatted[:5]

[{'date': '2017-04-29',
  'document_type': 'article',
  'headline': 'Fact-Checking President Trump Through His First 100 Days',
  'id': '5904573e7c459f24986dcd81',
  'source': 'The New York Times'},
 {'date': '2017-04-10',
  'document_type': 'article',
  'headline': 'Patients Prescribed Shelter and Medication Are Wary of Trump Cuts',
  'id': '58ec02327c459f24986d97e1',
  'source': 'The New York Times'},
 {'date': '2017-04-04',
  'document_type': 'article',
  'headline': 'Why an Open Market Won’t Repair American Health Care',
  'id': '58e37cc47c459f24986d83f7',
  'source': 'The New York Times'},
 {'date': '2017-03-22',
  'document_type': 'article',
  'headline': 'As Rivals Stand Silent, One Health Insurer Protests G.O.P. Plan',
  'id': '58d2d3ee7c459f247805d64d',
  'source': 'The New York Times'},
 {'date': '2017-03-17',
  'document_type': 'article',
  'headline': 'The Cost Can Be Debated, but Meals on Wheels Gets Results',
  'id': '58cc39fe7c459f247805c820',
  'source': 'The New York T

In [101]:
keys = all_formatted[1]
with open('all-formated.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(all_formatted)

In [102]:
import pandas as pd
import numpy as np

In [105]:
df = pd.read_csv('all-formated.csv')
df[:5]

,id,headline,date,source,document_type
0,5904573e7c459f24986dcd81,Fact-Checking President Trump Through His Firs...,2017-04-29,The New York Times,article
1,58ec02327c459f24986d97e1,Patients Prescribed Shelter and Medication Are...,2017-04-10,The New York Times,article
2,58e37cc47c459f24986d83f7,Why an Open Market Won’t Repair American Healt...,2017-04-04,The New York Times,article
3,58d2d3ee7c459f247805d64d,"As Rivals Stand Silent, One Health Insurer Pro...",2017-03-22,The New York Times,article
4,58cc39fe7c459f247805c820,"The Cost Can Be Debated, but Meals on Wheels G...",2017-03-17,The New York Times,article
